<a href="https://colab.research.google.com/github/nan-park/section2_project/blob/main/data_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [2]:
from google.colab import files
files.upload();
# model_data.csv
# model_data_stationary.csv (model_data.csv를 정상성 갖도록 변환한 데이터)

Saving model_data.csv to model_data.csv
Saving model_data_stationary.csv to model_data_stationary.csv


In [3]:
url1 = 'model_data.csv'
url2 = 'model_data_stationary.csv'
df = pd.read_csv(url1, index_col=0)
df_stationary = pd.read_csv(url2, index_col=0)

# **데이터 모델링**

먼저 시계열 데이터임을 무시하고, 각각의 시계열 데이터(row)는 다른 시간대 데이터와 독립적이라고 가정하고 모델링을 해보려고 한다.<br>
즉, 자기상관관계(autocorrelation)를 갖지 않는다고 가정한다.<br>
그러면 일반적 선형회귀나 랜덤포레스트, XGBRegressor 등을 사용할 수 있다.

타겟 설정 -> 소비자물가상승률(%)

In [5]:
target = '소비자물가상승률(%)'
features = df.columns.tolist()
features.remove(target)

X = df[features]
y = df[target]

In [15]:
len(y)  # 데이터 개수 265개

265

훈련/테스트 비율을 8:2로 잡고, 
훈련 데이터를 cross validation(cv=5)으로 한다.(데이터 개수가 적음)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- 기준모델(Baseline model)

타겟의 평균값을 기준모델로 잡는다.

In [12]:
# y_val_pred_base = [y_val.mean()] * len(y_val)
# # 회귀 평가지표
# # from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# print('validation data eval(Baseline)')
# print(f'mae : {mean_absolute_error(y_val, y_val_pred_base).round(5)}')
# print(f'mse : {mean_squared_error(y_val, y_val_pred_base).round(5)}')
# print(f'r2 score : {r2_score(y_val, y_val_pred_base)}')

validation data eval(Baseline)
mae : 1.07964
mse : 1.78366
r2 score : 0.0


- 다중선형회귀모델(1차)

In [31]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score

# 회귀평가지표: neg_mean_absolute_error(-0.4727), r2(0.7), neg_mean_squared_error(-0.49253)
linear = LinearRegression(normalize=True)
linear_mae_score = cross_val_score(linear, X_train, y_train, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5).mean()
print(f'Linear model(baseline) mae score : {linear_mae_score.round(5)}')

Linear model(baseline) mae score : -0.54727


- 랜덤포레스트 회귀(RandomForestRegressor)

In [35]:
from sklearn.ensemble import RandomForestRegressor
randomforest = RandomForestRegressor( # 하이퍼파라미터 모두 default
    n_estimators=100,
    criterion='absolute_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    n_jobs=-1,
    random_state=42,
    verbose=3
)
random_mae_score = cross_val_score(randomforest, X_train, y_train, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5).mean()
print(f'RandomForest model mae score : {random_mae_score.round(5)}')

RandomForest model mae score : -0.37418


 - XGBRegressor 회귀

In [37]:
from xgboost import XGBRegressor
xgb = XGBRegressor(
    booster='gblinear',
    objective='reg:linear',
    eval_metric=mean_absolute_error,
    max_depth=6,
    learning_rate=0.01,
    n_jobs=-1,
)
xgb_mae_score = cross_val_score(xgb, X_train, y_train, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5).mean()
print(f'XGB model mae score : {xgb_mae_score.round(5)}')

XGB model mae score : nan


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py", line 396, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 216, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 74,